In [1]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>

<h1 id="tocheading">Содержание</h1>
<div id="toc"></div>

## Необходимые модули

In [2]:
from graphframes import *
import arrayProcessing
import pyspark

## Подготовка датасета


### Предобработка

In [3]:
arrayProcessing.prepare_dataset()
unique_ids = list(arrayProcessing.get_unique_ids())

Теперь есть два "сырых" датасета:

**"./data/train.txt"** - обучающая выборка

**"./data/test.txt"** - тестовая выборка

### Создание датасета для обучения

In [4]:
from pyspark import SparkContext
from pyspark.sql import SQLContext, Window
from pyspark.sql.types import *
from pyspark.sql.functions import rank, col, monotonically_increasing_id, udf
from random import shuffle
from pyspark.sql import SparkSession
from math import log

In [5]:
if 'sc' not in locals():
    sc = SparkContext()
if 'sqlContext' not in locals():
    scqContext = SQLContext(sc)
if 'spark' not in locals():
    spark = SparkSession.builder.appName('task2').getOrCreate()

In [6]:
train_list = open("./data/train.txt", "r").read().split("\n")
train_list = [edge.split(" ") for edge in train_list]
train_list = [(int(edge[0]), int(edge[1])) for edge in train_list]

edges = spark.createDataFrame(train_list, ["src", "dst"])
vertices = spark.createDataFrame([(i, i) for i in unique_ids], ["id", "name"])

In [7]:
g = GraphFrame(vertices, edges)

In [8]:
connections = g.find("(a)-[]->(b); (b)-[]->(c); !(a)-[]->(c); !(c)-[]->(a)")
connections = connections.drop(connections.b)

In [9]:
connections = connections.sort("a")
connections = connections.withColumn("idx", monotonically_increasing_id())
window = Window.partitionBy(connections["a"]).orderBy(connections["idx"])
connections = connections.select('*', rank().over(window).alias('num'))\
                         .filter(col('num') <= 40)
connections = connections.drop(connections.idx)\
                         .drop(connections.num)

In [10]:
connections = connections.withColumn("a", connections["a"].getItem("id"))
connections = connections.withColumn("c", connections["c"].getItem("id"))

In [11]:
friend_list = {}

shuffled_train = train_list.copy()
shuffle(shuffled_train)

for (user, friend) in shuffled_train:
    if user not in friend_list:
        friend_list[user] = []
    if friend not in friend_list:
        friend_list[friend] = []
    friend_list[user].append(friend)
    friend_list[friend].append(user)

In [12]:
def _between_friends(u1, u2):
    return len(set(friend_list[u1]).intersection(set(friend_list[u2])))

In [13]:
diff_between_friends_udf = udf(diff_between_friends)

In [16]:
data = connections.withColumn("cn", diff_between_friends_udf(connections.a, connections.c))
data = data.withColumn("cn", data["cn"].cast(IntegerType()))

In [ ]:
data.write.format("com.databricks.spark.csv").save("./data/data_cn.csv")

In [22]:
data.coalesce(1).write.csv("./data/data_cn.csv")

KeyboardInterrupt: 

In [65]:
data.show()

Py4JJavaError: An error occurred while calling o431.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 385.0 failed 1 times, most recent failure: Lost task 0.0 in stage 385.0 (TID 15376, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 377, in main
    process()
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 372, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 345, in dump_stream
    self.serializer.dump_stream(self._batched(iterator), stream)
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 141, in dump_stream
    for obj in iterator:
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 334, in _batched
    for item in iterato

In [33]:
df = connections.sample(False, 0.00001, 228)

In [41]:
df = df.withColumn("a", df["a"].getItem("id"))
df = df.withColumn("c", df["c"].getItem("id"))

+-----+--------------+----+
|    a|             c|rank|
+-----+--------------+----+
|  598|[27431, 27431]|  29|
|  834|[19755, 19755]|  26|
| 2118|[27642, 27642]|  29|
| 2252|[15197, 15197]|  10|
|21741|[39058, 39058]|  30|
|22354|[25256, 25256]|   2|
|23396|[36770, 36770]|   4|
|28237|[32797, 32797]|  38|
|30304|[42247, 42247]|   5|
+-----+--------------+----+



In [13]:
connections = connections.filter("a.id == 1")

In [16]:
f = open("./data/unique_ids.txt", "w")
f.write("\n".join([str(x) for x in unique_ids]))
f.close()

In [49]:
a = {1:1, 2:1, 3:2, 4:3, 5:5, 6:8}

l = list(map(lambda x: a[x], [i for i in range(1, 7, 1)]))

In [21]:
f = open("./data/data_cn.csv", "r")

IsADirectoryError: [Errno 21] Is a directory: './data/data_cn.csv'